In [0]:
orders = spark.read.json('/public/retail_db_json/orders')

In [0]:
import getpass
username = getpass.getuser()

- Partition data by year, month and then day_of_month

In [0]:
from pyspark.sql.functions import *

In [0]:
orders.\
    withColumn('year', date_format('order_date', 'yyyy')).\
    withColumn('month', date_format('order_date','MM')).\
    withColumn('day_of_month', date_format('order_date','dd')).\
    show()

+-----------------+--------------------+--------+---------------+----+-----+------------+
|order_customer_id|          order_date|order_id|   order_status|year|month|day_of_month|
+-----------------+--------------------+--------+---------------+----+-----+------------+
|            11599|2013-07-25 00:00:...|       1|         CLOSED|2013|   07|          25|
|              256|2013-07-25 00:00:...|       2|PENDING_PAYMENT|2013|   07|          25|
|            12111|2013-07-25 00:00:...|       3|       COMPLETE|2013|   07|          25|
|             8827|2013-07-25 00:00:...|       4|         CLOSED|2013|   07|          25|
|            11318|2013-07-25 00:00:...|       5|       COMPLETE|2013|   07|          25|
|             7130|2013-07-25 00:00:...|       6|       COMPLETE|2013|   07|          25|
|             4530|2013-07-25 00:00:...|       7|       COMPLETE|2013|   07|          25|
|             2911|2013-07-25 00:00:...|       8|     PROCESSING|2013|   07|          25|
|         

In [0]:
dbutils.fs.rm(f'/user/{username}/retail_db/orders_partitioned', recurse=True)

False

In [0]:
orders.\
    withColumn('year', date_format('order_date', 'yyyy')).\
    withColumn('month', date_format('order_date','MM')).\
    withColumn('day_of_month', date_format('order_date','dd')).\
    coalesce(1).\
    write.\
    partitionBy('year','month','day_of_month').\
    parquet(f'/user/{username}/retail_db/orders_partitioned')

In [0]:
dbutils.fs.ls(f'/user/{username}/retail_db/orders_partitioned')

[FileInfo(path='dbfs:/user/root/retail_db/orders_partitioned/_SUCCESS', name='_SUCCESS', size=0, modificationTime=1698240639000),
 FileInfo(path='dbfs:/user/root/retail_db/orders_partitioned/year=2013/', name='year=2013/', size=0, modificationTime=1698240570000),
 FileInfo(path='dbfs:/user/root/retail_db/orders_partitioned/year=2014/', name='year=2014/', size=0, modificationTime=1698240574000)]

In [0]:
dbutils.fs.ls(f'/user/{username}/retail_db/orders_partitioned/year=2013')

[FileInfo(path='dbfs:/user/root/retail_db/orders_partitioned/year=2013/month=07/', name='month=07/', size=0, modificationTime=1698240570000),
 FileInfo(path='dbfs:/user/root/retail_db/orders_partitioned/year=2013/month=08/', name='month=08/', size=0, modificationTime=1698240570000),
 FileInfo(path='dbfs:/user/root/retail_db/orders_partitioned/year=2013/month=09/', name='month=09/', size=0, modificationTime=1698240571000),
 FileInfo(path='dbfs:/user/root/retail_db/orders_partitioned/year=2013/month=10/', name='month=10/', size=0, modificationTime=1698240572000),
 FileInfo(path='dbfs:/user/root/retail_db/orders_partitioned/year=2013/month=11/', name='month=11/', size=0, modificationTime=1698240573000),
 FileInfo(path='dbfs:/user/root/retail_db/orders_partitioned/year=2013/month=12/', name='month=12/', size=0, modificationTime=1698240574000)]

In [0]:
dbutils.fs.ls(f'/user/{username}/retail_db/orders_partitioned/year=2014/month=01/day_of_month=09')

[FileInfo(path='dbfs:/user/root/retail_db/orders_partitioned/year=2014/month=01/day_of_month=09/_SUCCESS', name='_SUCCESS', size=0, modificationTime=1698240638000),
 FileInfo(path='dbfs:/user/root/retail_db/orders_partitioned/year=2014/month=01/day_of_month=09/_committed_165562728816696533', name='_committed_165562728816696533', size=125, modificationTime=1698240637000),
 FileInfo(path='dbfs:/user/root/retail_db/orders_partitioned/year=2014/month=01/day_of_month=09/_started_165562728816696533', name='_started_165562728816696533', size=0, modificationTime=1698240598000),
 FileInfo(path='dbfs:/user/root/retail_db/orders_partitioned/year=2014/month=01/day_of_month=09/part-00000-tid-165562728816696533-92724667-03ad-4c11-ae61-4a4e045f0519-431-163.c000.snappy.parquet', name='part-00000-tid-165562728816696533-92724667-03ad-4c11-ae61-4a4e045f0519-431-163.c000.snappy.parquet', size=3333, modificationTime=1698240598000)]

In [0]:
spark.read.parquet(f'/user/{username}/retail_db/orders_partitioned').count()

68883